In [1]:
from nemoguardrails import LLMRails, RailsConfig

from dotenv import load_dotenv
import os 

import openai
from langchain_openai import ChatOpenAI

load_dotenv()

model = ChatOpenAI(model="gpt-4o")

In [2]:
# API Configuration
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
import nest_asyncio
nest_asyncio.apply()

# Load NeMo Guardrails configuration
config = RailsConfig.from_path("C:/Users/dangn/Phu/HDSI_Capstone/final-project/config")
rails = LLMRails(config)

c:\Users\dangn\Phu\HDSI_Capstone\final-project\db_venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 4611.15it/s]


In [4]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.postgres import PostgresSaver
from langgraph.graph import START, MessagesState, StateGraph
from psycopg import Connection

In [5]:
# Database connection details
DB_URI = os.getenv("DB_URI")

connection_kwargs = {
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": "5432",
    "autocommit": True,
    "prepare_threshold": 0,
}

In [6]:
# wrapper function for LLM response with persistent state management

def get_response(state: MessagesState):

    system_prompt = (
        "You are a helpful smart home analyst conversing with the homeowner directly."
        "Be precise, concise, and keep the response within a comfortable length for the homeowner to read conveniently and understand their utility usage."
        "Give actionable insights to pinpoint inefficiencies and provide suggestions to lower utility cost."
    )

    # incorporate only the latest 5 messages + first message
    # trimmed_messages = state['messages'][0] + state['messages'][-5:]

    # construct prompt
    prompt = [SystemMessage(content=system_prompt)] + state['messages']

    res = model.invoke(prompt)

    print(f"\n{res}\n")
    
    return {"messages": res}


In [ ]:
# Define LangGraph workflow with persistence and guardrails
workflow = StateGraph(state_schema=MessagesState)

# Add the node and edge to the workflow
workflow.add_node("model", get_response)
workflow.add_edge(START, "model")

def test_func(state: MessagesState):
    print("THIS IS A TESTTT!")

workflow.add_node("test", test_func)
workflow.add_edge("model", "test")

In [17]:
test_prompt = """
    This is a time-series dataset of utility usage for a smart home:

    The dataset contains the following columns:
    use, gen, Dishwasher, Furnace_1, Furnace_2, Home_office, Fridge, Wine_cellar, Garage_door, Kitchen_12, Kitchen_14, Kitchen_38, Barn, Well, Microwave, Living_room, kitchen, Furnace

    Column description (everything in kilowatt):
    "use" contains total energy consumption
    "gen" contains total energy generated by means of solar or other power generation resources
    The rest of the columns either contain energy consumed by a specific appliance or room in the house

    Summary usage by month:
                    use       gen  Dishwasher  Furnace_1  Furnace_2  Home_office    Fridge  Wine_cellar  Garage_door  Kitchen_12  Kitchen_14  Kitchen_38      Barn      Well  Microwave  Living_room   kitchen   Furnace
    2015-12-31  0.836898  0.003448    0.000091   0.238546   0.253805     0.054039  0.060343     0.007168     0.013105    0.000596    0.000156    0.000014  0.034275  0.001016   0.004256     0.001633  0.000765  0.492352
    2016-01-31  1.047812  0.059034    0.039739   0.221878   0.234376     0.091420  0.055713     0.018900     0.013647    0.002872    0.007600    0.000010  0.074052  0.017327   0.009432     0.044356  0.010482  0.456255
    2016-02-29  0.995173  0.058891    0.031232   0.222302   0.234220     0.069926  0.053357     0.019491     0.013786    0.002907    0.006851    0.000011  0.069036  0.017568   0.007959     0.034542  0.009769  0.456522
    2016-03-31  0.770586  0.083415    0.034448   0.118393   0.173024     0.080134  0.053053     0.029050     0.013790    0.000656    0.008312    0.000011  0.040879  0.016721   0.008835     0.035553  0.008979  0.291417
    2016-04-30  0.690589  0.089860    0.030955   0.094915   0.122734     0.071471  0.057580     0.036019     0.014573    0.000613    0.006815    0.000010  0.061524  0.014753   0.011446     0.035707  0.007438  0.217649
    2016-05-31  0.570911  0.082698    0.026486   0.048589   0.085656     0.062082  0.059366     0.036092     0.013750    0.000808    0.005549    0.000010  0.075403  0.009305   0.009396     0.024818  0.006366  0.134245
    2016-06-30  0.537512  0.106605    0.026522   0.025971   0.067307     0.077959  0.069597     0.046525     0.014561    0.000836    0.005723    0.000009  0.071232  0.019061   0.008029     0.027586  0.006568  0.093277
    2016-07-31  1.165928  0.091958    0.026554   0.060952   0.109885     0.093516  0.079352     0.064768     0.014563    0.002790    0.006819    0.000009  0.051821  0.017802   0.011128     0.023662  0.009618  0.170838
    2016-08-31  1.400862  0.091221    0.031419   0.069776   0.127411     0.099653  0.084534     0.098990     0.013831    0.004962    0.007028    0.000009  0.036223  0.014269   0.013601     0.040737  0.011999  0.197187
    2016-09-30  0.722752  0.082260    0.030318   0.030985   0.076128     0.082817  0.074198     0.063424     0.014452    0.005606    0.006822    0.000009  0.062955  0.013834   0.010799     0.033432  0.012437  0.107113
    2016-10-31  0.682110  0.054308    0.030859   0.046078   0.103116     0.088901  0.061057     0.030076     0.014488    0.003710    0.007460    0.000009  0.061934  0.014195   0.014059     0.032818  0.011179  0.149193
    2016-11-30  0.842199  0.051300    0.032169   0.112768   0.138107     0.082647  0.056133     0.028830     0.014302    0.004504    0.008514    0.000009  0.053323  0.018005   0.011663     0.051442  0.013027  0.250875
    2016-12-31  0.934789  0.040754    0.033800   0.191171   0.213707     0.075729  0.051394     0.024994     0.014050    0.003543    0.007113    0.000009  0.034346  0.015905   0.009571     0.051298  0.010664  0.404878

    Sample data:
    use,gen,Dishwasher,Furnace_1,Furnace_2,Home_office,Fridge,Wine_cellar,Garage_door,Kitchen_12,Kitchen_14,Kitchen_38,Barn,Well,Microwave,Living_room,kitchen,Furnace
    0.6316,0.003466667,3.33e-05,0.158883333,0.0639,0.0434,0.0825,0.009083333,0.012983333,0.000516667,5e-05,0.0,0.031766667,0.001033333,0.004083333,0.001616667,0.000566667,0.22278333299999997
    1.070016667,0.0035,0.000233333,0.497633333,0.0629,0.043583333,0.170883333,0.008883333,0.0125,0.00035,0.000216667,1.67e-05,0.0317,0.001083333,0.004083333,0.001683333,0.000583367,0.560533333
    1.01625,0.003466667,0.000216667,0.466683333,0.062766667,0.043533333,0.14955,0.0089,0.012566667,0.000416667,0.0002,1.67e-05,0.031733333,0.001016667,0.004083333,0.001683333,0.000633367,0.52945
    0.820666667,0.003466667,0.000216667,0.44695,0.062916667,0.043583333,0.0105,0.007733333,0.012616667,0.000733333,0.000333333,1.67e-05,0.031766667,0.001016667,0.0041,0.0016,0.0010833659999999999,0.509866667
    0.539166667,0.003516667,3.33e-05,0.100133333,0.116283333,0.043366667,0.007616667,0.007083333,0.013133333,0.000683333,5e-05,0.0,0.0318,0.001016667,0.004083333,0.001616667,0.0007333330000000001,0.216416666

    Please answer the following questions:
    1. Identify any sources with an increasing trend in electricity usage over time. 
    2. Highlight any significant outliers or anomalies.
"""

In [18]:
config = RailsConfig.from_path("C:/Users/dangn/Phu/HDSI_Capstone/final-project/config")
rails = LLMRails(config)

In [19]:
# open database connection + test

with Connection.connect(DB_URI, **connection_kwargs) as conn:
    checkpointer = PostgresSaver(conn)

    # Compile workflow with persistence
    app = workflow.compile(checkpointer=checkpointer)

    # user_prompt = "how can I save energy use at home?"
    
    # Step 1: run input moderation before sending the user prompt to the LLM
    # check_res = rails.generate(messages=[{
    #     "role": "user",
    #     "content": test_prompt}])

    # check_res = rails.generate(test_prompt)


    # print(f"Pass input moderation: {check_res}")
    # info = rails.explain()
    # info.print_llm_calls_summary()


    # TEST
    # test = rails.generate("how can I save energy at home?")
    # print(f"Pass output moderation: {test}")
    # info.print_llm_calls_summary()


    # if "yes" in check_res.lower():
    # specify configuration and input
    config = {"configurable": {"thread_id": "5"}}
    input_data = {"messages": [("human", test_prompt)]}

    # run the workflow
    response = app.invoke(input_data, config)
    print("Workflow response:", response)

    # retrieve the checkpoint tuple
    checkpoint_tuple = checkpointer.get_tuple(config)
    
    # else:
    # print("I cannot assist with your inquiry.")

    print("Workflow Response:", response)
    print("Checkpoint Tuple:", checkpoint_tuple)



content="Analyzing your smart home utility data, here are some insights:\n\n1. **Sources with Increasing Trend**:\n   - **Home Office**: There's a clear increase in energy consumption, growing from 0.054 kW in December 2015 to 0.099 kW in August 2016, indicating more device usage or prolonged activity.\n   - **Wine Cellar**: Usage rose steadily from 0.007 kW in December 2015 to 0.099 kW in August 2016, possibly due to higher cooling requirements or inefficiencies.\n   - **Microwave**: Shows a gradual increase from 0.004 kW in December 2015 to 0.014 kW by October 2016, suggesting increased duration or frequency of use.\n   - **Fridge**: Notable peak in consumption during August 2016 at 0.085 kW, which could be related to higher external temperatures.\n\n2. **Significant Outliers or Anomalies**:\n   - **August 2016**: Total usage spikes to 1.400 kW, with significant contributions from Home Office, Wine Cellar, and Fridge, potentially due to increased appliance use or operational ineffic

TypeError: test_func() takes 0 positional arguments but 1 was given

In [11]:
# inspect NeMo
info = rails.explain()
info.print_llm_calls_summary()

Summary: 3 LLM call(s) took 1.16 seconds and used 480 tokens.

1. Task `self_check_input` took 0.44 seconds and used 236 tokens.
2. Task `general` took 0.41 seconds and used 94 tokens.
3. Task `self_check_output` took 0.30 seconds and used 150 tokens.



In [12]:
print(info.llm_calls[2].prompt)


[cyan]User[/]
Your task is to check if the bot message below complies with chatbot regulations.

Company policy for the bot:
- Messages should not contain explicit content, even if just a few words.
- Messages should not contain abusive language or offensive content, even if just a few words.
- Messages should not contain any harmful content.
- Messages should not contain racially insensitive content.
- Messages should not contain any word that can be considered offensive.
- If a message is a refusal, it should be polite.
- It's okay to give instructions to users on how to use the bot or how to uphold chatbot regulations.

Bot message: "Yes"

Question: Should the message be blocked? (Yes or No)
Answer:


In [13]:
print(info.llm_calls[0].completion)

No


In [14]:
# inspect LangGraph response

print("""To analyze the dataset for trends and anomalies, let's first address the two questions:\n\n1. **Sources with Increasing Trend**:\n   - **Home Office**: There's a noticeable increase in usage starting from January 2016 (0.054 kW) peaking to higher values, like 0.099 kW in August 2016, and generally staying higher than the December 2015 value.\n   - **Wine Cellar**: Usage significantly increases, particularly noticeable from March 2016 (0.029 kW) to August 2016 (0.099 kW).\n   - **Fridge**: There seems to be increased usage over the months with the peak in July 2016 (0.079 kW) and August 2016 (0.085 kW).\n   - **Microwave**: Displayed increasing usage towards the end of the period, noticeable from 0.004 kW in December 2015 to 0.014 kW in October 2016. \n\n2. **Significant Outliers or Anomalies**:\n   - **July 2016**: Extremely high total usage of 1.165 kW with considerable increases in Home Office, Wine Cellar, and Fridge, which could suggest prolonged equipment use or an event requiring excessive energy.\n   - **August 2016**: Highest consumption of 1.4 kW stands out, driven by continued high usage in areas like Fridge and Wine Cellar.\n\nFor **actionable insights**:\n- **Fridge & Wine Cellar**: Consider inspecting these appliances for efficiency issues or ensure that they are only running when needed. These could have higher consumption due to operational inefficiencies or temperature setting issues.\n- **Home Office**: If possible, try scheduling high-energy usage activities during periods of peak solar generation to offset costs.\n- **Microwave**: Monitor usage times, reduce unnecessary long periods of operation by leveraging more efficient cooking practices.\n\nBy addressing these areas and monitoring the usage patterns, you could better optimize your energy usage and potentially see a reduction in your utility expenses.""")

To analyze the dataset for trends and anomalies, let's first address the two questions:

1. **Sources with Increasing Trend**:
   - **Home Office**: There's a noticeable increase in usage starting from January 2016 (0.054 kW) peaking to higher values, like 0.099 kW in August 2016, and generally staying higher than the December 2015 value.
   - **Wine Cellar**: Usage significantly increases, particularly noticeable from March 2016 (0.029 kW) to August 2016 (0.099 kW).
   - **Fridge**: There seems to be increased usage over the months with the peak in July 2016 (0.079 kW) and August 2016 (0.085 kW).
   - **Microwave**: Displayed increasing usage towards the end of the period, noticeable from 0.004 kW in December 2015 to 0.014 kW in October 2016. 

2. **Significant Outliers or Anomalies**:
   - **July 2016**: Extremely high total usage of 1.165 kW with considerable increases in Home Office, Wine Cellar, and Fridge, which could suggest prolonged equipment use or an event requiring excessiv

In [13]:
# function to get checkpoint

def get_checkpoint(id):
    
    with Connection.connect(DB_URI, **connection_kwargs) as conn:
        config = {"configurable": {"thread_id": str(id)}}
        checkpointer = PostgresSaver(conn)
        checkpoint_tuple = checkpointer.get_tuple(config)
        print("Checkpoint Tuple:", checkpoint_tuple)

In [14]:
get_checkpoint(5)

Checkpoint Tuple: CheckpointTuple(config={'configurable': {'thread_id': '5', 'checkpoint_ns': '', 'checkpoint_id': '1eff2403-8b5e-6d94-8007-e0912ac4c444'}}, checkpoint={'v': 1, 'id': '1eff2403-8b5e-6d94-8007-e0912ac4c444', 'ts': '2025-02-23T23:45:11.505038+00:00', 'pending_sends': [], 'versions_seen': {'model': {'start:model': '00000000000000000000000000000008.0.78534776715067'}, '__input__': {}, '__start__': {'__start__': '00000000000000000000000000000007.0.6852427880925172'}}, 'channel_versions': {'model': '00000000000000000000000000000009.0.6053688574565573', 'messages': '00000000000000000000000000000009.0.0291753458090811', '__start__': '00000000000000000000000000000008.0.24629125056190093', 'start:model': '00000000000000000000000000000009.0.5826676357939201'}, 'channel_values': {'model': 'model', 'messages': [HumanMessage(content='\n    This is a time-series dataset of utility usage for a smart home:\n\n    The dataset contains the following columns:\n    use, gen, Dishwasher, Fur

##### sanity check

Checkpoint Tuple: CheckpointTuple(config={'configurable': {'thread_id': '5', 'checkpoint_ns': '', 'checkpoint_id': '1efe1fbb-f310-6c52-8001-9f0547961a7c'}}, checkpoint={'v': 1, 'id': '1efe1fbb-f310-6c52-8001-9f0547961a7c', 'ts': '2025-02-03T06:54:43.247829+00:00', 'pending_sends': [], 'versions_seen': {'model': {'start:model': '00000000000000000000000000000002.0.8505414510573167'}, '__input__': {}, '__start__': {'__start__': '00000000000000000000000000000001.0.2124756237778367'}}, 'channel_versions': {'model': '00000000000000000000000000000003.0.9656790533439723', 'messages': '00000000000000000000000000000003.0.21247764883461817', '__start__': '00000000000000000000000000000002.0.16660232414228993', 'start:model': '00000000000000000000000000000003.0.33132537851380817'}, 'channel_values': {'model': 'model', 'messages': [HumanMessage(content='\n    This is a time-series dataset of utility usage for a smart home:\n\n    The dataset contains the following columns:\n    use, gen, Dishwasher, Furnace_1, Furnace_2, Home_office, Fridge, Wine_cellar, Garage_door, Kitchen_12, Kitchen_14, Kitchen_38, Barn, Well, Microwave, Living_room, kitchen, Furnace\n\n    Column description (everything in kilowatt):\n    "use" contains total energy consumption\n    "gen" contains total energy generated by means of solar or other power generation resources\n    The rest of the columns either contain energy consumed by a specific appliance or room in the house\n\n    Summary usage by month:\n                    use       gen  Dishwasher  Furnace_1  Furnace_2  Home_office    Fridge  Wine_cellar  Garage_door  Kitchen_12  Kitchen_14  Kitchen_38      Barn      Well  Microwave  Living_room   kitchen   Furnace\n    2015-12-31  0.836898  0.003448    0.000091   0.238546   0.253805     0.054039  0.060343     0.007168     0.013105    0.000596    0.000156    0.000014  0.034275  0.001016   0.004256     0.001633  0.000765  0.492352\n    2016-01-31  1.047812  0.059034    0.039739   0.221878   0.234376     0.091420  0.055713     0.018900     0.013647    0.002872    0.007600    0.000010  0.074052  0.017327   0.009432     0.044356  0.010482  0.456255\n    2016-02-29  0.995173  0.058891    0.031232   0.222302   0.234220     0.069926  0.053357     0.019491     0.013786    0.002907    0.006851    0.000011  0.069036  0.017568   0.007959     0.034542  0.009769  0.456522\n    2016-03-31  0.770586  0.083415    0.034448   0.118393   0.173024     0.080134  0.053053     0.029050     0.013790    0.000656    0.008312    0.000011  0.040879  0.016721   0.008835     0.035553  0.008979  0.291417\n    2016-04-30  0.690589  0.089860    0.030955   0.094915   0.122734     0.071471  0.057580     0.036019     0.014573    0.000613    0.006815    0.000010  0.061524  0.014753   0.011446     0.035707  0.007438  0.217649\n    2016-05-31  0.570911  0.082698    0.026486   0.048589   0.085656     0.062082  0.059366     0.036092     0.013750    0.000808    0.005549    0.000010  0.075403  0.009305   0.009396     0.024818  0.006366  0.134245\n    2016-06-30  0.537512  0.106605    0.026522   0.025971   0.067307     0.077959  0.069597     0.046525     0.014561    0.000836    0.005723    0.000009  0.071232  0.019061   0.008029     0.027586  0.006568  0.093277\n    2016-07-31  1.165928  0.091958    0.026554   0.060952   0.109885     0.093516  0.079352     0.064768     0.014563    0.002790    0.006819    0.000009  0.051821  0.017802   0.011128     0.023662  0.009618  0.170838\n    2016-08-31  1.400862  0.091221    0.031419   0.069776   0.127411     0.099653  0.084534     0.098990     0.013831    0.004962    0.007028    0.000009  0.036223  0.014269   0.013601     0.040737  0.011999  0.197187\n    2016-09-30  0.722752  0.082260    0.030318   0.030985   0.076128     0.082817  0.074198     0.063424     0.014452    0.005606    0.006822    0.000009  0.062955  0.013834   0.010799     0.033432  0.012437  0.107113\n    2016-10-31  0.682110  0.054308    0.030859   0.046078   0.103116     0.088901  0.061057     0.030076     0.014488    0.003710    0.007460    0.000009  0.061934  0.014195   0.014059     0.032818  0.011179  0.149193\n    2016-11-30  0.842199  0.051300    0.032169   0.112768   0.138107     0.082647  0.056133     0.028830     0.014302    0.004504    0.008514    0.000009  0.053323  0.018005   0.011663     0.051442  0.013027  0.250875\n    2016-12-31  0.934789  0.040754    0.033800   0.191171   0.213707     0.075729  0.051394     0.024994     0.014050    0.003543    0.007113    0.000009  0.034346  0.015905   0.009571     0.051298  0.010664  0.404878\n\n    Sample data:\n    use,gen,Dishwasher,Furnace_1,Furnace_2,Home_office,Fridge,Wine_cellar,Garage_door,Kitchen_12,Kitchen_14,Kitchen_38,Barn,Well,Microwave,Living_room,kitchen,Furnace\n    0.6316,0.003466667,3.33e-05,0.158883333,0.0639,0.0434,0.0825,0.009083333,0.012983333,0.000516667,5e-05,0.0,0.031766667,0.001033333,0.004083333,0.001616667,0.000566667,0.22278333299999997\n    1.070016667,0.0035,0.000233333,0.497633333,0.0629,0.043583333,0.170883333,0.008883333,0.0125,0.00035,0.000216667,1.67e-05,0.0317,0.001083333,0.004083333,0.001683333,0.000583367,0.560533333\n    1.01625,0.003466667,0.000216667,0.466683333,0.062766667,0.043533333,0.14955,0.0089,0.012566667,0.000416667,0.0002,1.67e-05,0.031733333,0.001016667,0.004083333,0.001683333,0.000633367,0.52945\n    0.820666667,0.003466667,0.000216667,0.44695,0.062916667,0.043583333,0.0105,0.007733333,0.012616667,0.000733333,0.000333333,1.67e-05,0.031766667,0.001016667,0.0041,0.0016,0.0010833659999999999,0.509866667\n    0.539166667,0.003516667,3.33e-05,0.100133333,0.116283333,0.043366667,0.007616667,0.007083333,0.013133333,0.000683333,5e-05,0.0,0.0318,0.001016667,0.004083333,0.001616667,0.0007333330000000001,0.216416666\n\n    Please answer the following questions:\n    1. Identify any sources with an increasing trend in electricity usage over time. \n    2. Highlight any significant outliers or anomalies.\n', additional_kwargs={}, response_metadata={}, id='4724c413-739a-4492-afee-0a53397f8c6f'), AIMessage(content="To analyze the dataset for trends and anomalies, let's first address the two questions:\n\n1. **Sources with Increasing Trend**:\n   - **Home Office**: There's a noticeable increase in usage starting from January 2016 (0.054 kW) peaking to higher values, like 0.099 kW in August 2016, and generally staying higher than the December 2015 value.\n   - **Wine Cellar**: Usage significantly increases, particularly noticeable from March 2016 (0.029 kW) to August 2016 (0.099 kW).\n   - **Fridge**: There seems to be increased usage over the months with the peak in July 2016 (0.079 kW) and August 2016 (0.085 kW).\n   - **Microwave**: Displayed increasing usage towards the end of the period, noticeable from 0.004 kW in December 2015 to 0.014 kW in October 2016. \n\n2. **Significant Outliers or Anomalies**:\n   - **July 2016**: Extremely high total usage of 1.165 kW with considerable increases in Home Office, Wine Cellar, and Fridge, which could suggest prolonged equipment use or an event requiring excessive energy.\n   - **August 2016**: Highest consumption of 1.4 kW stands out, driven by continued high usage in areas like Fridge and Wine Cellar.\n\nFor **actionable insights**:\n- **Fridge & Wine Cellar**: Consider inspecting these appliances for efficiency issues or ensure that they are only running when needed. These could have higher consumption due to operational inefficiencies or temperature setting issues.\n- **Home Office**: If possible, try scheduling high-energy usage activities during periods of peak solar generation to offset costs.\n- **Microwave**: Monitor usage times, reduce unnecessary long periods of operation by leveraging more efficient cooking practices.\n\nBy addressing these areas and monitoring the usage patterns, you could better optimize your energy usage and potentially see a reduction in your utility expenses.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 430, 'prompt_tokens': 2423, 'total_tokens': 2853, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-7e0ba032-3209-46ca-9e84-41fb97d00456-0', usage_metadata={'input_tokens': 2423, 'output_tokens': 430, 'total_tokens': 2853, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}, metadata={'step': 1, 'source': 'loop', 'writes': {'model': {'messages': AIMessage(content="To analyze the dataset for trends and anomalies, let's first address the two questions:\n\n1. **Sources with Increasing Trend**:\n   - **Home Office**: There's a noticeable increase in usage starting from January 2016 (0.054 kW) peaking to higher values, like 0.099 kW in August 2016, and generally staying higher than the December 2015 value.\n   - **Wine Cellar**: Usage significantly increases, particularly noticeable from March 2016 (0.029 kW) to August 2016 (0.099 kW).\n   - **Fridge**: There seems to be increased usage over the months with the peak in July 2016 (0.079 kW) and August 2016 (0.085 kW).\n   - **Microwave**: Displayed increasing usage towards the end of the period, noticeable from 0.004 kW in December 2015 to 0.014 kW in October 2016. \n\n2. **Significant Outliers or Anomalies**:\n   - **July 2016**: Extremely high total usage of 1.165 kW with considerable increases in Home Office, Wine Cellar, and Fridge, which could suggest prolonged equipment use or an event requiring excessive energy.\n   - **August 2016**: Highest consumption of 1.4 kW stands out, driven by continued high usage in areas like Fridge and Wine Cellar.\n\nFor **actionable insights**:\n- **Fridge & Wine Cellar**: Consider inspecting these appliances for efficiency issues or ensure that they are only running when needed. These could have higher consumption due to operational inefficiencies or temperature setting issues.\n- **Home Office**: If possible, try scheduling high-energy usage activities during periods of peak solar generation to offset costs.\n- **Microwave**: Monitor usage times, reduce unnecessary long periods of operation by leveraging more efficient cooking practices.\n\nBy addressing these areas and monitoring the usage patterns, you could better optimize your energy usage and potentially see a reduction in your utility expenses.", additional_kwargs={'refusal': None}, response_metadata={'logprobs': None, 'model_name': 'gpt-4o-2024-08-06', 'token_usage': {'total_tokens': 2853, 'prompt_tokens': 2423, 'completion_tokens': 430, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'finish_reason': 'stop', 'system_fingerprint': 'fp_50cad350e4'}, id='run-7e0ba032-3209-46ca-9e84-41fb97d00456-0', usage_metadata={'input_tokens': 2423, 'output_tokens': 430, 'total_tokens': 2853, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})}}, 'parents': {}, 'thread_id': '5'}, parent_config={'configurable': {'thread_id': '5', 'checkpoint_ns': '', 'checkpoint_id': '1efe1fbb-7fcd-6e14-8000-56a86be166e5'}}, pending_writes=[])

In [ ]:
from langgraph.checkpoint.postgres import PostgresSaver
from langgraph.graph import Message
from langchain.schema import AIMessage, HumanMessage
from langchain_core.runnables import ConfigurableFieldSpec
from sqlalchemy import create_engine

# Initialize DB Connection
engine = create_engine(DB_URI)
connection_kwargs = {}  # Add specific kwargs if needed

with engine.connect() as conn:
    checkpointer = PostgresSaver(conn)

    # Define user prompt and LLM response
    user_prompt = "how can I save energy use at home?"
    llm_response = "You can save energy by using LED lighting, smart thermostats, and reducing phantom loads."

    # Define a thread ID (Ensure this is unique per session)
    thread_id = "5"

    # Create a message list to mimic LangGraph's format
    messages = [
        Message(content=HumanMessage(content=user_prompt)),
        Message(content=AIMessage(content=llm_response))
    ]

    # Save to LangGraph's persistence layer
    checkpointer.put_tuple(config={"configurable": {"thread_id": thread_id}}, value=messages)

    # Retrieve the stored conversation
    saved_interaction = checkpointer.get_tuple({"configurable": {"thread_id": thread_id}})
    print("Retrieved Conversation:", saved_interaction)


In [ ]:
# open database connection + test

with Connection.connect(DB_URI, **connection_kwargs) as conn:
    checkpointer = PostgresSaver(conn)

    # Compile workflow with persistence
    app = workflow.compile(checkpointer=checkpointer)

    # user_prompt = "how can I save energy use at home?"

    # specify configuration and input
    config = {"configurable": {"thread_id": "5"}}
    input_data = {"messages": [("human", test_prompt)]}

    # run the workflow
    response = app.invoke(input_data, config)
    print("Workflow response:", response)

    # retrieve the checkpoint tuple
    checkpoint_tuple = checkpointer.get_tuple(config)
        
    # else:
    #     print("I cannot assist with your inquiry.")

In [151]:
# confirm persistence

with Connection.connect(DB_URI, **connection_kwargs) as conn:
    checkpointer = PostgresSaver(conn)

    # compile the workflow with the Postgres checkpointer
    app = workflow.compile(checkpointer=checkpointer)

    config = {"configurable": {"thread_id": "5"}}
    input_data = {"messages": [("human", "What did I just ask you?")]}

    response = app.invoke(input_data, config)
    print("Workflow Response:", response)

    checkpoint_tuple = checkpointer.get_tuple(config)


content='You asked me to identify any sources with an increasing trend in electricity usage over time from a time-series dataset of utility usage for a smart home. Additionally, you wanted me to highlight any significant outliers or anomalies present in the data.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 2867, 'total_tokens': 2915, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 2688}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'stop', 'logprobs': None} id='run-54201e70-f5ef-4444-9e3c-9822804f09a6-0' usage_metadata={'input_tokens': 2867, 'output_tokens': 48, 'total_tokens': 2915, 'input_token_details': {'audio': 0, 'cache_read': 2688}, 'output_token_details': {'audio': 0, 'reasoning': 0}}

Workflow Response: {'messa